# 2.5 Hypothesis Testing 

In this notebook I will redo all of my hypothesis testing work to expand the scope and usefulness of identified features. I will remove those features with pvalue < 0.05 and build a linear regression to predict % close. I will also build a logistic regression to predict the sign(%close). 

#### Steps

1) Load a dataset  
2) Perform transformations  
3) Test features and remove, retest  
4) Build regressions  

In [1]:
from sine_modules import *
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)

In [2]:
pd.__version__

'1.1.1'

In [27]:
stock = 'RRR'
data_dir = './data/prepared/august25screenfixed/'
suffix = '.pickle'

df = load_set(stock, data_dir, suffix)

In [21]:
df.head()

,open,high,low,close,volume,datetime,date,hour,minute,min_num,SYMBOL,prev_close,diff_1,pct_change,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_maxcov,open_sin,time,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_maxcov,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_maxcov,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,%close,mesa_close,close_amp,close_omega,close_phase,close_offset,close_freq,close_period,close_maxcov,close_sin,close_angle,close_rad,close_rad2,close_sin2,close_cos2,close_cos,close_tan,close_tan2,close_xsinx,close_xcosx,close_sinxcosx,close_xsinxcosx,close_xsinx2,close_xcosx2,close_sinxcosx2,close_xsinxcosx2,close_xtanx,close_xtanx2,decision,D2
8845,17.28,17.28,17.28,17.28,100,2020-08-25 22:48:00,2020-08-25,22,48,1368,RRR,17.30,-0.02,-0.001156,-0.001157,-0.001156,0.490588,0.001894,2.999950,141.151213,-0.000400,0.477457,2.094430,2.221737e+05,-0.001211,8845,26675.705720,3.584091,4.061547,-0.001907,-0.605856,-0.903685,0.473836,1.313140,0.000001,0.001045,0.001095,-0.000001,2.204898e-06,0.000700,0.001156,-1.335851e-06,-0.000548,-0.001518,-0.001156,0.490591,-0.001940,2.802241,-332.993391,-0.000401,0.445991,2.242200,2.440428e+05,0.001482,24452.828207,4.954177,5.400168,0.001098,0.634822,0.239439,-4.054941,-1.217125,-0.000002,-0.000277,0.000355,-4.103498e-07,-0.000001,-0.000734,0.000697,-8.057384e-07,0.004688,0.001407,-0.001156,0.490588,0.002575,2.990564,224.104713,-0.000389,0.475963,2.101003,1.440349e+05,-0.001352,26675.646515,3.524886,4.000850,-0.002340,-0.653000,-0.927438,0.403237,1.159812,0.000002,0.001072,0.001254,-0.000001,0.000003,0.000755,0.001528,-0.000002,-0.000466,-0.001341,-0.001156,0.490591,-0.002573,2.833824,-611.936535,-0.000390,0.451017,2.217211,1.726659e+05,0.001653,24453.239631,5.365601,5.816618,0.000767,0.893118,0.607740,-1.306702,-0.503655,-0.000002,-0.000703,0.001004,-0.000001,-8.865898e-07,-0.001033,0.000685,-7.918291e-07,0.001511,0.000582,0.0,0.0
8844,17.30,17.30,17.30,17.30,200,2020-08-25 22:44:00,2020-08-25,22,44,1364,RRR,17.44,-0.14,-0.008028,-0.008060,-0.002882,0.489934,0.001881,1.287313,-270.196033,-0.000346,0.204882,4.880852,2.322576e+05,-0.000633,8844,11114.801657,6.130034,6.334916,-0.000249,0.998662,0.988295,-0.154360,0.051777,0.000002,-0.002848,-0.000626,0.000002,7.176933e-07,-0.002878,-0.000249,7.167332e-07,0.000445,-0.000149,-0.008028,0.488493,-0.002108,2.808354,-387.061715,-0.000335,0.446963,2.237320,1.819360e+05,-0.002103,24450.020219,2.146189,2.593152,-0.001434,-0.853339,-0.544164,-1.541776,-0.610962,0.000017,0.004368,0.001145,-9.187537e-06,0.000012,0.006850,0.001223,-9.820883e-06,0.012377,0.004905,-0.002882,0.489934,0.002457,3.001820,124.667322,-0.000327,0.477755,2.093125,2.107880e+05,0.001151,26672.767225,0.645596,1.123351,0.001888,0.432664,0.798741,0.753279,2.083731,-0.000003,-0.002302,0.000919,-0.000003,-0.000005,-0.001247,0.000817,-0.000002,-0.002171,-0.006005,-0.008028,0.488493,-0.002719,2.838034,-649.214703,-0.000325,0.451687,2.213922,1.408935e+05,-0.001990,24450.356426,2.482395,2.934083,-0.000885,-0.978547,-0.790484,-0.774819,-0.210541,0.000016,0.006346,0.001573,-0.000013,7.103888e-06,0.007855,0.000866,-6.951488e-06,0.006220,0.001690,0.0,0.0
8843,17.35,17.44,17.35,17.44,25,2020-08-25 22:36:00,2020-08-25,22,36,1356,RRR,17.35,0.09,0.005187,0.005174,0.000000,0.496402,0.004198,3.106833,-802.280898,-0

In [5]:
df['datetime'].shift(-1)

8845   2020-08-25 22:44:00
8844   2020-08-25 22:36:00
8843   2020-08-25 22:34:00
8842   2020-08-25 22:28:00
8841   2020-08-25 22:12:00
               ...        
4      2020-07-27 13:32:00
3      2020-07-27 13:31:00
2      2020-07-27 13:30:00
1      2020-07-27 12:42:00
0                      NaT
Name: datetime, Length: 8846, dtype: datetime64[ns]

In [7]:
df['open'].rolling(30).mean()

8845          NaN
8844          NaN
8843          NaN
8842          NaN
8841          NaN
          ...    
4       10.602963
3       10.609980
2       10.616647
1       10.624980
0       10.639313
Name: open, Length: 8846, dtype: float64

In [8]:
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=2)
df['open'].rolling(indexer).mean()

8845    17.290
8844    17.325
8843    17.350
8842    17.335
8841    17.320
         ...  
4       10.655
3       10.700
2       10.785
1       10.925
0          NaN
Name: open, Length: 8846, dtype: float64

In [28]:
cols = ['open','close','high','low']

# sma, ema, high, low
# diff from 
# (actual price - high/low/sma/ema) / high/low/sma/ema

stat_cols = ['_sma30',
            '_sma50',
            '_std30',
            '_std50',
            '_var30',
            '_var50',
            '_min30',
            '_min50',
            '_max30',
            '_max50',
            '_skew30',
            '_skew50',
            '_kurt30',
            '_kurt50',]
#             '_ewmmean30',
#             '_ewmmean50']


for col in cols:
    for i in range(1,61):
        df[f'%{col}_{i}'] = df[f'%{col}'].shift(-i)
    
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=30)
    
    df[f'{col}_sma30'] = df[col].rolling(indexer).mean()
    df[f'{col}_std30'] = df[col].rolling(indexer).std()
    df[f'{col}_var30'] = df[col].rolling(indexer).var()
    df[f'{col}_min30'] = df[col].rolling(indexer).min()
    df[f'{col}_max30'] = df[col].rolling(indexer).max()
    df[f'{col}_skew30'] = df[col].rolling(indexer).skew()
    df[f'{col}_kurt30'] = df[col].rolling(indexer).kurt()
    #df[f'{col}_ewmmean30'] = df[col].ewm(indexer).mean()
    
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=50)
    
    df[f'{col}_sma50'] = df[col].rolling(indexer).mean()
    df[f'{col}_std50'] = df[col].rolling(indexer).std()
    df[f'{col}_var50'] = df[col].rolling(indexer).var()
    df[f'{col}_min50'] = df[col].rolling(indexer).min()
    df[f'{col}_max50'] = df[col].rolling(indexer).max()
    df[f'{col}_skew50'] = df[col].rolling(indexer).skew()
    df[f'{col}_kurt50'] = df[col].rolling(indexer).kurt()
    #df[f'{col}_ewmmean50'] = df[col].ewm(indexer).mean()

    for stat_col in stat_cols:
        df[f'%{col}{stat_col}'] = (df[f'{col}'] - df[f'{col}{stat_col}']) / df[f'{col}{stat_col}']
        
        
df.head()

,open,high,low,close,volume,datetime,date,hour,minute,min_num,SYMBOL,prev_close,diff_1,pct_change,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_maxcov,open_sin,time,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_maxcov,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_maxcov,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,...,%high_std30,%high_std50,%high_var30,%high_var50,%high_min30,%high_min50,%high_max30,%high_max50,%high_skew30,%high_skew50,%high_kurt30,%high_kurt50,%low_1,%low_2,%low_3,%low_4,%low_5,%low_6,%low_7,%low_8,%low_9,%low_10,%low_11,%low_12,%low_13,%low_14,%low_15,%low_16,%low_17,%low_18,%low_19,%low_20,%low_21,%low_22,%low_23,%low_24,%low_25,%low_26,%low_27,%low_28,%low_29,%low_30,%low_31,%low_32,%low_33,%low_34,%low_35,%low_36,%low_37,%low_38,%low_39,%low_40,%low_41,%low_42,%low_43,%low_44,%low_45,%low_46,%low_47,%low_48,%low_49,%low_50,%low_51,%low_52,%low_53,%low_54,%low_55,%low_56,%low_57,%low_58,%low_59,%low_60,low_sma30,low_std30,low_var30,low_min30,low_max30,low_skew30,low_kurt30,low_sma50,low_std50,low_var50,low_min50,low_max50,low_skew50,low_kurt50,%low_sma30,%low_sma50,%low_std30,%low_std50,%low_var30,%low_var50,%low_min30,%low_min50,%low_max30,%low_max50,%low_skew30,%low_skew50,%low_kurt30,%low_kurt50
8845,17.28,17.28,17.28,17.28,100,2020-08-25 22:48:00,2020-08-25,22,48,1368,RRR,17.30,-0.02,-0.001156,-0.001157,-0.001156,0.490588,0.001894,2.999950,141.151213,-0.000400,0.477457,2.094430,2.221737e+05,-0.001211,8845,26675.705720,3.584091,4.061547,-0.001907,-0.605856,-0.903685,0.473836,1.313140,0.000001,0.001045,0.001095,-0.000001,2.204898e-06,0.000700,0.001156,-1.335851e-06,-0.000548,-0.001518,-0.001156,0.490591,-0.001940,2.802241,-332.993391,-0.000401,0.445991,2.242200,2.440428e+05,0.001482,24452.828207,4.954177,5.400168,0.001098,0.634822,0.239439,-4.054941,-1.217125,-0.000002,-0.000277,0.000355,-4.103498e-07,-0.000001,-0.000734,0.000697,-8.057384e-07,0.004688,0.001407,-0.001156,0.490588,0.002575,2.990564,224.104713,-0.000389,0.475963,2.101003,1.440349e+05,-0.001352,26675.646515,3.524886,4.000850,-0.002340,-0.653000,-0.927438,0.403237,1.159812,0.000002,0.001072,0.001254,-0.000001,0.000003,0.000755,0.001528,-0.000002,-0.000466,-0.001341,...,175.770098,74.255262,1807.314088,326.740421,0.004651,0.025519,-0.026479,-0.026479,9.107055,-35.842614,1.618333,-22.554117,-0.002882,0.000000,0.001732,0.000000,0.011092,-0.006381,-0.000580,-0.007480,0.004624,0.000000,0.000000,-0.003456,-0.001151,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.001149,-0.001148,-0.001147,0.000000,-0.000573,0.000000,0.000000,0.004028,0.000000,0.001152,0.003468,-0.009164,0.015707,0.015957,0.000000,0.004154,0.000000,-0.003548,0.003561,-0.003253,0.000000,0.000888,-0.001183,-0.000591,-0.000591,-0.000295,0.000591,0.001183,0.001184,0.000889,0.000000,-0.000888,0.000296,-0.000296,0.000889,0.000593,-0.000296,0.000000,0.000297,-0.002662,17.355000,0.071715,0.005143,17.13,17.45,-1.093830,1.963110,17.2173,0.218411,0.047703,16.85,17.46,-0.677560,-1.307059,-0.004322,0.003642,239.952318,78.116968,3358.839088,361.239276,0.008757,0.025519,-0.009742,-0.010309,-16.797696,-26.503292,7.802358,-14.220519
8844,17.30,17.30,17.30,17.30,200,2020-08-25 22:44:00,2020-08-25,22,44,1364,RRR,17.44,-0.14,-0.008028,-0.008060,-0.002882,0.489934,0.001881,1.287313,-270.196033,-0.000346,0.204882,4

In [29]:
drop_cols = ['datetime',
            'date',
            'min_num',
            'SYMBOL',
            'prev_close',
            'diff_1',
            'time',
            'decision',
            'open_maxcov',
            'high_maxcov',
            'low_maxcov',
            'close_maxcov',
            'pct_change',
            'D2']

df.drop(drop_cols, axis=1, inplace=True)

df['target'] = df['%close'].shift(1)

df.dropna(axis=0, inplace=True)

In [33]:
y = df['target']
X = df.drop('target', axis=1)

In [34]:
y.shape, X.shape

((8782,), (8782, 468))

In [35]:
X.head()

,open,high,low,close,volume,hour,minute,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_sin,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,%close,mesa_close,close_amp,close_omega,close_phase,close_offset,close_freq,close_period,close_sin,close_angle,close_rad,...,%high_std30,%high_std50,%high_var30,%high_var50,%high_min30,%high_min50,%high_max30,%high_max50,%high_skew30,%high_skew50,%high_kurt30,%high_kurt50,%low_1,%low_2,%low_3,%low_4,%low_5,%low_6,%low_7,%low_8,%low_9,%low_10,%low_11,%low_12,%low_13,%low_14,%low_15,%low_16,%low_17,%low_18,%low_19,%low_20,%low_21,%low_22,%low_23,%low_24,%low_25,%low_26,%low_27,%low_28,%low_29,%low_30,%low_31,%low_32,%low_33,%low_34,%low_35,%low_36,%low_37,%low_38,%low_39,%low_40,%low_41,%low_42,%low_43,%low_44,%low_45,%low_46,%low_47,%low_48,%low_49,%low_50,%low_51,%low_52,%low_53,%low_54,%low_55,%low_56,%low_57,%low_58,%low_59,%low_60,low_sma30,low_std30,low_var30,low_min30,low_max30,low_skew30,low_kurt30,low_sma50,low_std50,low_var50,low_min50,low_max50,low_skew50,low_kurt50,%low_sma30,%low_sma50,%low_std30,%low_std50,%low_var30,%low_var50,%low_min30,%low_min50,%low_max30,%low_max50,%low_skew30,%low_skew50,%low_kurt30,%low_kurt50
8844,17.30,17.30,17.30,17.30,200,22,44,-0.008060,-0.002882,0.489934,0.001881,1.287313,-270.196033,-0.000346,0.204882,4.880852,-0.000633,11114.801657,6.130034,6.334916,-0.000249,0.998662,0.988295,-0.154360,0.051777,0.000002,-0.002848,-0.000626,0.000002,7.176933e-07,-0.002878,-0.000249,7.167332e-07,0.000445,-0.000149,-0.008028,0.488493,-0.002108,2.808354,-387.061715,-0.000335,0.446963,2.237320,-0.002103,24450.020219,2.146189,2.593152,-0.001434,-0.853339,-0.544164,-1.541776,-0.610962,0.000017,0.004368,0.001145,-0.000009,0.000012,0.006850,0.001223,-0.000010,0.012377,0.004905,-0.002882,0.489934,0.002457,3.001820,124.667322,-0.000327,0.477755,2.093125,0.001151,26672.767225,0.645596,1.123351,0.001888,0.432664,0.798741,0.753279,2.083731,-0.000003,-0.002302,0.000919,-0.000003,-0.000005,-0.001247,0.000817,-0.000002,-0.002171,-0.006005,-0.008028,0.488493,-0.002719,2.838034,-649.214703,-0.000325,0.451687,2.213922,-0.001990,24450.356426,2.482395,...,179.099649,72.732683,1873.906572,313.249046,0.005814,0.026706,-0.025352,-0.025352,9.016392,-40.993553,1.473616,-19.012349,0.000000,0.001732,0.000000,0.011092,-0.006381,-0.000580,-0.007480,0.004624,0.000000,0.000000,-0.003456,-0.001151,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.001149,-0.001148,-0.001147,0.000000,-0.000573,0.000000,0.000000,0.004028,0.000000,0.001152,0.003468,-0.009164,0.015707,0.015957,0.000000,0.004154,0.000000,-0.003548,0.003561,-0.003253,0.000000,0.000888,-0.001183,-0.000591,-0.000591,-0.000295,0.000591,0.001183,0.001184,0.000889,0.000000,-0.000888,0.000296,-0.000296,0.000889,0.000593,-0.000296,0.000000,0.000297,-0.002662,-0.000887,17.358333,0.070422,0.004959,17.13,17.45,-1.261216,2.567249,17.2094,0.223188,0.049813,16.85,17.46,-0.594950,-1.450096,-0.003361,0.005265,244.663420,76.513117,3487.469116,346.299609,0.009924,0.026706,-0.008596,-0.009164,-14.716923,-30.078059,5.738731,-12.930245
8843,17.35,17.44,17.35,17.44,25,22,36,0.005174,0.000000,0.496402,0.004198,3.106833,-802.280898,-0.000187,0.494468,2.022376,-0.002822,26671.442713,5.604270,6.098

In [36]:
X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()

In [37]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     4.231
Date:                Mon, 07 Sep 2020   Prob (F-statistic):          7.81e-157
Time:                        18:48:37   Log-Likelihood:                 39380.
No. Observations:                8782   AIC:                        -7.784e+04
Df Residuals:                    8321   BIC:                        -7.457e+04
Df Model:                         460                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -6.5229      1.133     -5.757      0.000      -8.744      -4.302
open                 0.0156      0.017      0.917      0.359      -0.018       0.049
high                -0.0752      0.018     -4.184      0.000      -0.110      -0.040
low                  0.0445      0.018      2.510      0.012       0.010       0.079
close                0.0100      0.018      0.567      0.571      -0.025       0.045
volume            6.103e-09   2.19e-09      2.785      0.005    1.81e-09    1.04e-08
hour              -3.26e-05    2.2e-05     -1.480      0.139   -7.58e-05    1.06e-05
minute            -2.07e-06    1.9e-06     -1.089      0.276   -5.79e-06    1.65e-06
log_return         -20.8773      3.387     -6.164      0.000     -27.517     -14.237
%open               -4.4601      1.708     -2.611      0.009      -7.808      -1.112
mesa_open           14.6226      2.478      5.901      0.000       9.765      19.480
open_amp             0.0002      0.000      1.315      0.189      -0.000       0.001
open_omega           0.0001   8.47e-05      1.331      0.183   -5.33e-05       0.000
open_phase       -8.053e-08   9.74e-08     -0.827      0.408   -2.71e-07     1.1e-07
open_offset       8.381e-05      0.000      0.441      0.659      -0.000       0.000
open_freq         1.583e-05   1.35e-05      1.173      0.241   -1.06e-05    4.23e-05
open_period      -1.676e-09   3.42e-07     -0.005      0.996   -6.73e-07    6.69e-07
open_sin             0.0036      0.031      0.117      0.907      -0.057       0.064
open_angle       -1.824e-08    1.7e-08     -1.074      0.283   -5.15e-08    1.51e-08
open_rad          2.872e-06   1.36e-05      0.212      0.832   -2.37e-05    2.94e-05
open_rad2         2.325e-05   1.29e-05      1.803      0.071   -2.03e-06    4.85e-05
open_sin2            0.0011      0.002      0.641      0.522      -0.002       0.005
open_cos2           -0.0004      0.000     -1.876      0.061      -0.001     1.9e-05
open_cos             0.0003      0.000      1.403      0.161      -0.000       0.001
open_tan          3.228e-07   2.77e-07      1.167      0.243    -2.2e-07    8.65e-07
open_tan2         7.344e-09   4.82e-07      0.015      0.988   -9.38e-07    9.53e-07
open_xsinx          13.1883      5.763      2.288      0.022       1.890      24.486
open_xcosx          -0.0096      0.058     -0.167      0.868      -0.123       0.104
open_sinxcosx       -0.0967      0.045     -2.145      0.032      -0.185      -0.008
open_xsinxcosx     -13.6373      6.854     -1.990      0.047     -27.073      -0.202
open_xsinx2         -1.2843      0.243     -5.291      0.000      -1.760      -0.809
open_xcosx2          0.0697      0.055      1.262      0.207      -0.039       0.178
open_sinxcosx2      -0.0017      0.002     -0.814      0.416      -0.006       0.002
open_xsinxcosx2     -1.

In [40]:
pmask = results.pvalues < 0.05

pcols = X.columns[pmask]

X2 = X[pcols]

X2.shape

In [58]:
model2 = sm.OLS(y,X2)
results2 = model2.fit()

In [59]:
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.110
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     6.548
Date:                Mon, 07 Sep 2020   Prob (F-statistic):          3.54e-123
Time:                        18:54:08   Log-Likelihood:                 38969.
No. Observations:                8782   AIC:                        -7.761e+04
Df Residuals:                    8618   BIC:                        -7.645e+04
Df Model:                         163                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -5.2438      0.686     -7.644      0.000      -6.589      -3.899
high                -0.0023      0.002     -1.058      0.290      -0.007       0.002
low                  0.0005      0.003      0.189      0.850      -0.005       0.006
volume            6.585e-09   2.19e-09      3.013      0.003     2.3e-09    1.09e-08
log_return         -27.9917      2.285    -12.249      0.000     -32.471     -23.512
%open               -0.0152      0.059     -0.258      0.796      -0.131       0.100
mesa_open            0.4321      0.195      2.221      0.026       0.051       0.813
open_xsinx          -4.1100      3.291     -1.249      0.212     -10.562       2.342
open_sinxcosx       -0.0584      0.036     -1.621      0.105      -0.129       0.012
open_xsinxcosx      -7.5926      6.052     -1.254      0.210     -19.457       4.272
open_xsinx2         -1.4485      0.223     -6.489      0.000      -1.886      -1.011
open_xsinxcosx2     -1.1776      0.273     -4.309      0.000      -1.713      -0.642
open_xtanx2         -0.0002   7.35e-05     -3.227      0.001      -0.000   -9.32e-05
high_phase        -2.27e-07   8.98e-08     -2.528      0.011   -4.03e-07    -5.1e-08
high_angle        1.474e-08   7.07e-09      2.084      0.037    8.72e-10    2.86e-08
high_sin2           -0.0045      0.002     -2.575      0.010      -0.008      -0.001
high_cos2           -0.0003      0.000     -1.477      0.140      -0.001    8.55e-05
high_cos             0.0002      0.000      1.035      0.301      -0.000       0.001
high_xcosx           0.1088      0.056      1.945      0.052      -0.001       0.218
high_xsinxcosx      45.7985      6.908      6.630      0.000      32.258      59.339
high_xsinx2          0.5122      0.127      4.042      0.000       0.264       0.761
high_xcosx2         -0.1260      0.054     -2.323      0.020      -0.232      -0.020
high_xsinxcosx2      1.0348      0.181      5.712      0.000       0.680       1.390
high_xtanx2         -0.0002   8.29e-05     -2.076      0.038      -0.000   -9.57e-06
%low                 0.3242      0.068      4.755      0.000       0.191       0.458
mesa_low            -1.2468      0.205     -6.078      0.000      -1.649      -0.845
low_sin              0.1366      0.027      5.145      0.000       0.085       0.189
low_tan2         -8.803e-07   2.37e-07     -3.713      0.000   -1.35e-06   -4.16e-07
low_xcosx            0.1144      0.056      2.042      0.041       0.005       0.224
low_xsinx2           1.0239      0.261      3.922      0.000       0.512       1.536
low_xcosx2          -0.0785      0.055     -1.440      0.150      -0.185       0.028
low_xsinxcosx2       1.9422      0.326      5.954      0.000       1.303       2.582
low_xtanx2           0.0006      0.000      4.000      0.000       0.000       0.001
%close              21.

In [60]:
pmask2 = results2.pvalues < 0.05

pcols2 = X2.columns[pmask2]

X3 = X2[pcols2]

X3.shape

(8782, 53)

In [61]:
model3 = sm.OLS(y,X3)
results3 = model3.fit()

In [62]:
results3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     15.50
Date:                Mon, 07 Sep 2020   Prob (F-statistic):          4.30e-129
Time:                        18:54:49   Log-Likelihood:                 38844.
No. Observations:                8782   AIC:                        -7.758e+04
Df Residuals:                    8729   BIC:                        -7.721e+04
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -6.0809      0.659     -9.223      0.000      -7.373      -4.789
volume            5.912e-09   2.16e-09      2.736      0.006    1.68e-09    1.01e-08
log_return         -29.2787      2.188    -13.380      0.000     -33.568     -24.989
mesa_open            0.1232      0.034      3.661      0.000       0.057       0.189
open_xsinx2         -1.3547      0.219     -6.190      0.000      -1.784      -0.926
open_xsinxcosx2     -1.0642      0.255     -4.175      0.000      -1.564      -0.565
open_xtanx2         -0.0003    7.3e-05     -3.527      0.000      -0.000      -0.000
high_phase       -2.126e-07   9.01e-08     -2.359      0.018   -3.89e-07    -3.6e-08
high_angle        1.415e-08   7.03e-09      2.015      0.044    3.83e-10    2.79e-08
high_sin2           -0.0050      0.002     -2.914      0.004      -0.008      -0.002
high_xsinxcosx      42.5923      6.603      6.451      0.000      29.650      55.535
high_xsinx2          0.5275      0.125      4.235      0.000       0.283       0.772
high_xcosx2         -0.0223      0.021     -1.063      0.288      -0.063       0.019
high_xsinxcosx2      1.0014      0.179      5.594      0.000       0.651       1.352
high_xtanx2         -0.0002    8.3e-05     -2.059      0.039      -0.000   -8.23e-06
%low                 0.2865      0.044      6.549      0.000       0.201       0.372
mesa_low            -0.8710      0.106     -8.199      0.000      -1.079      -0.663
low_sin              0.1388      0.026      5.346      0.000       0.088       0.190
low_tan2         -8.844e-07   2.38e-07     -3.718      0.000   -1.35e-06   -4.18e-07
low_xcosx            0.0390      0.021      1.864      0.062      -0.002       0.080
low_xsinx2           0.9911      0.259      3.831      0.000       0.484       1.498
low_xsinxcosx2       1.8303      0.324      5.641      0.000       1.194       2.466
low_xtanx2           0.0006      0.000      3.937      0.000       0.000       0.001
%close              22.2281      1.628     13.654      0.000      19.037      25.419
mesa_close          13.1329      1.341      9.792      0.000      10.504      15.762
close_sin2           0.0055      0.002      3.423      0.001       0.002       0.009
close_xcosx         -0.2620      0.046     -5.723      0.000      -0.352      -0.172
close_xsinxcosx    -25.4801      5.943     -4.288      0.000     -37.129     -13.831
close_xcosx2         0.2191      0.046      4.805      0.000       0.130       0.309
close_xsinxcosx2    -1.5033      0.278     -5.409      0.000      -2.048      -0.958
%open_4              0.0279      0.012      2.333      0.020       0.004       0.051
%open_9              0.0312      0.011      2.754      0.006       0.009       0.053
%open_41            -0.0040      0.011     -0.364      0.716      -0.026       0.018
%open_42             0.

In [63]:
pmask3 = results3.pvalues < 0.05

pcols3 = X3.columns[pmask3]

X4 = X3[pcols3]

X4.shape

(8782, 44)

In [65]:
model4 = sm.OLS(y,X4)
results4 = model4.fit()
results4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     18.46
Date:                Mon, 07 Sep 2020   Prob (F-statistic):          3.11e-132
Time:                        18:57:03   Log-Likelihood:                 38839.
No. Observations:                8782   AIC:                        -7.759e+04
Df Residuals:                    8738   BIC:                        -7.728e+04
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -6.0814      0.657     -9.249      0.000      -7.370      -4.793
volume            5.848e-09   2.14e-09      2.735      0.006    1.66e-09       1e-08
log_return         -29.3181      2.188    -13.399      0.000     -33.607     -25.029
mesa_open            0.1232      0.034      3.662      0.000       0.057       0.189
open_xsinx2         -1.3830      0.218     -6.338      0.000      -1.811      -0.955
open_xsinxcosx2     -1.0617      0.255     -4.167      0.000      -1.561      -0.562
open_xtanx2         -0.0003    7.3e-05     -3.450      0.001      -0.000      -0.000
high_phase       -2.213e-07      9e-08     -2.459      0.014   -3.98e-07   -4.49e-08
high_angle        5.204e-09    5.3e-09      0.981      0.326   -5.19e-09    1.56e-08
high_sin2           -0.0051      0.002     -2.972      0.003      -0.008      -0.002
high_xsinxcosx      41.6483      6.549      6.359      0.000      28.810      54.487
high_xsinx2          0.5602      0.121      4.610      0.000       0.322       0.798
high_xsinxcosx2      1.0035      0.179      5.613      0.000       0.653       1.354
high_xtanx2         -0.0002    8.3e-05     -2.018      0.044      -0.000    -4.8e-06
%low                 0.2813      0.044      6.442      0.000       0.196       0.367
mesa_low            -0.8599      0.106     -8.110      0.000      -1.068      -0.652
low_sin              0.1383      0.026      5.335      0.000       0.088       0.189
low_tan2         -8.985e-07   2.38e-07     -3.779      0.000   -1.36e-06   -4.32e-07
low_xsinx2           0.9644      0.258      3.737      0.000       0.458       1.470
low_xsinxcosx2       1.8445      0.323      5.702      0.000       1.210       2.479
low_xtanx2           0.0006      0.000      4.033      0.000       0.000       0.001
%close              22.2735      1.627     13.694      0.000      19.085      25.462
mesa_close          13.1219      1.338      9.810      0.000      10.500      15.744
close_sin2           0.0056      0.002      3.484      0.000       0.002       0.009
close_xcosx         -0.2511      0.045     -5.520      0.000      -0.340      -0.162
close_xsinxcosx    -26.4626      5.914     -4.475      0.000     -38.055     -14.870
close_xcosx2         0.2121      0.045      4.707      0.000       0.124       0.300
close_xsinxcosx2    -1.5590      0.277     -5.624      0.000      -2.102      -1.016
%open_4              0.0281      0.012      2.354      0.019       0.005       0.052
%open_9              0.0318      0.011      2.812      0.005       0.010       0.054
open_kurt50      -9.359e-05   2.26e-05     -4.148      0.000      -0.000   -4.94e-05
%open_sma30         -0.1565      0.031     -5.017      0.000      -0.218      -0.095
%close_1            -3.2596      0.342     -9.537      0.000      -3.930      -2.590
%close_2            -3.

In [71]:
ys = y.copy()
ys

8844   -0.001156
8843   -0.008028
8842    0.005187
8841    0.001732
8840    0.000000
          ...   
65      0.000000
64     -0.000960
63      0.000000
62     -0.002871
61      0.003842
Name: target, Length: 8782, dtype: float64

In [81]:
ys[ys > 0] = 1
ys[ys < 0] = 0

In [82]:
ys

8844    0.0
8843    0.0
8842    1.0
8841    1.0
8840    0.0
       ... 
65      0.0
64      0.0
63      0.0
62      0.0
61      1.0
Name: target, Length: 8782, dtype: float64

In [86]:
X3.drop('const', axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [92]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X3, ys)
lr.score(X3, ys)

0.5616032794352084

In [93]:
ys.value_counts() / ys.shape[0]

0.0    0.560351
1.0    0.439649
Name: target, dtype: float64

In [105]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=True)
poly.fit(X3)
X3_poly = poly.transform(X3)
X3_poly = pd.DataFrame(X3_poly, index=X3.index, columns=poly.get_feature_names(X3.columns))

In [106]:
X3_poly

,1,volume,log_return,mesa_open,open_xsinx2,open_xsinxcosx2,open_xtanx2,high_phase,high_angle,high_sin2,high_xsinxcosx,high_xsinx2,high_xcosx2,high_xsinxcosx2,high_xtanx2,%low,mesa_low,low_sin,low_tan2,low_xcosx,low_xsinx2,low_xsinxcosx2,low_xtanx2,%close,mesa_close,close_sin2,close_xcosx,close_xsinxcosx,close_xcosx2,close_xsinxcosx2,%open_4,%open_9,%open_41,%open_42,%open_43,open_kurt50,%open_sma30,%close_1,%close_2,%close_3,%close_21,close_sma30,close_kurt50,%high_21,%high_22,%high_23,%high_24,%low_3,%low_22,%low_23,%low_28,low_sma30,%low_sma30,volume^2,volume log_return,volume mesa_open,volume open_xsinx2,volume open_xsinxcosx2,volume open_xtanx2,volume high_phase,volume high_angle,volume high_sin2,volume high_xsinxcosx,volume high_xsinx2,volume high_xcosx2,volume high_xsinxcosx2,volume high_xtanx2,volume %low,volume mesa_low,volume low_sin,volume low_tan2,volume low_xcosx,volume low_xsinx2,volume low_xsinxcosx2,volume low_xtanx2,volume %close,volume mesa_close,volume close_sin2,volume close_xcosx,volume close_xsinxcosx,volume close_xcosx2,volume close_xsinxcosx2,volume %open_4,volume %open_9,volume %open_41,volume %open_42,volume %open_43,volume open_kurt50,volume %open_sma30,volume %close_1,volume %close_2,volume %close_3,volume %close_21,volume close_sma30,volume close_kurt50,volume %high_21,volume %high_22,volume %high_23,volume %high_24,volume %low_3,...,%close_3 %high_22,%close_3 %high_23,%close_3 %high_24,%close_3 %low_3,%close_3 %low_22,%close_3 %low_23,%close_3 %low_28,%close_3 low_sma30,%close_3 %low_sma30,%close_21^2,%close_21 close_sma30,%close_21 close_kurt50,%close_21 %high_21,%close_21 %high_22,%close_21 %high_23,%close_21 %high_24,%close_21 %low_3,%close_21 %low_22,%close_21 %low_23,%close_21 %low_28,%close_21 low_sma30,%close_21 %low_sma30,close_sma30^2,close_sma30 close_kurt50,close_sma30 %high_21,close_sma30 %high_22,close_sma30 %high_23,close_sma30 %high_24,close_sma30 %low_3,close_sma30 %low_22,close_sma30 %low_23,close_sma30 %low_28,close_sma30 low_sma30,close_sma30 %low_sma30,close_kurt50^2,close_kurt50 %high_21,close_kurt50 %high_22,close_kurt50 %high_23,close_kurt50 %high_24,close_kurt50 %low_3,close_kurt50 %low_22,close_kurt50 %low_23,close_kurt50 %low_28,close_kurt50 low_sma30,close_kurt50 %low_sma30,%high_21^2,%high_21 %high_22,%high_21 %high_23,%high_21 %high_24,%high_21 %low_3,%high_21 %low_22,%high_21 %low_23,%high_21 %low_28,%high_21 low_sma30,%high_21 %low_sma30,%high_22^2,%high_22 %high_23,%high_22 %high_24,%high_22 %low_3,%high_22 %low_22,%high_22 %low_23,%high_22 %low_28,%high_22 low_sma30,%high_22 %low_sma30,%high_23^2,%high_23 %high_24,%high_23 %low_3,%high_23 %low_22,%high_23 %low_23,%high_23 %low_28,%high_23 low_sma30,%high_23 %low_sma30,%high_24^2,%high_24 %low_3,%high_24 %low_22,%high_24 %low_23,%high_24 %low_28,%high_24 low_sma30,%high_24 %low_sma30,%low_3^2,%low_3 %low_22,%low_3 %low_23,%low_3 %low_28,%low_3 low_sma30,%low_3 %low_sma30,%low_22^2,%low_22 %low_23,%low_22 %low_28,%low_22 low_sma30,%low_22 %low_sma30,%low_23^2,%low_23 %low_28,%low_23 low_sma30,%low_23 %low_sma30,%low_28^2,%low_28 low_sma30,%low_28 %low_sma30,low_sma30^2,low_sma30 %low_sma30,%low_sma30^2
8844,1.0,200.0,-0.008060,0.489934,7.176933e-07,7.167332e-07,-0.000149,-387.061715,24450.020219,-0.001434,-9.187537e-06,1.150878e-05,0.006850,-9.820883e-06,0.004905,-0.002882,0.489934,0.001151,2.083731,-0.002302,-0.000005,-2.354013e-06,-0.006005,-0.008028,0.488493,-0.000885,0.006346,-1.262718e-05,0.007855,-6.951488e-06,0.009324,0.000000,-0.000591,-0.000591,-0.000590,-1.423588,-0.003590,0.005187,0.001732,0.000000,-0.001148,17.374333,-0.995016,-0.001719,0.000000,0.000000,0.000000,0.000000,-0.001147,0.000000,0.000000,17.358333,-0.003361,40000.0,-1.611983,97.986868,0.000144,0.000143,-0.029843,-77412.342930,4.890004e+06,-0.286733,-0.001838,0.002302,1.370039,-0.001964,0.980902,-0.576369,97.986868,0.230218,416.746108,-0.460370,-0.001088,-0.000471,-1.200997,-1.605505,97.698573,-0.176988,1.269126,-0.002525,1.571062,-0.001390,1.86

In [111]:
model5 = sm.OLS(y, X3_poly)
results5 = model5.fit()
results5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.333
Method:                 Least Squares   F-statistic:                     15.93
Date:                Mon, 07 Sep 2020   Prob (F-statistic):               0.00
Time:                        19:25:28   Log-Likelihood:                 40386.
No. Observations:                8782   AIC:                        -8.018e+04
Df Residuals:                    8487   BIC:                        -7.809e+04
Df Model:                         294                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
1                                   -2.3133      0.532     -4.350      0.000      -3.356      -1.271
volume                            3.503e-05      0.000      0.350      0.727      -0.000       0.000
log_return                          -1.8935      0.462     -4.095      0.000      -2.800      -0.987
mesa_open                           -0.2665      0.482     -0.553      0.581      -1.212       0.679
open_xsinx2                         -1.1502      0.631     -1.823      0.068      -2.387       0.086
open_xsinxcosx2                     -0.1309      0.448     -0.293      0.770      -1.008       0.746
open_xtanx2                          1.0683      0.452      2.362      0.018       0.182       1.955
high_phase                           0.0024      0.004      0.605      0.545      -0.005       0.010
high_angle                           0.0011      0.000      3.807      0.000       0.001       0.002
high_sin2                            0.3237      0.458      0.706      0.480      -0.575       1.222
high_xsinxcosx                      -0.2167      0.061     -3.544      0.000      -0.337      -0.097
high_xsinx2                         -0.5835      0.176     -3.308      0.001      -0.929      -0.238
high_xcosx2                         -0.8728      0.536     -1.629      0.103      -1.923       0.178
high_xsinxcosx2                      0.7234      0.169      4.274      0.000       0.392       1.055
high_xtanx2                         -2.2450      0.409     -5.495      0.000      -3.046      -1.444
%low                                 0.5783      0.491      1.177      0.239      -0.385       1.541
mesa_low                             1.9659      0.351      5.596      0.000       1.277       2.655
low_sin                              1.0941      0.413      2.648      0.008       0.284       1.904
low_tan2                             0.2591      0.278      0.933      0.351      -0.286       0.804
low_xcosx                           -0.2915      0.534     -0.546      0.585      -1.338       0.755
low_xsinx2                          -1.5116      1.062     -1.424      0.154      -3.592       0.569
low_xsinxcosx2                       1.6606      0.896      1.854      0.064      -0.095       3.416
low_xtanx2                          -0.1535      0.273     -0.563      0.573      -0.688       0.381
%close                              -2.0797      0.537     -3.876      0.000      -3.132      -1.028
mesa_close                           3.3273      1.004      3.314      0.001       1.359       5.295
close_sin2                           0.7419      0.453      1.638      0.101      -0.146       1.629
close_xcosx                          0.6162      0.561      1.099      0.272      -0.483       1.715
close_xsinxcosx                     -0.0024      0.06

In [113]:
pmask5 = results5.pvalues < 0.05
pmask5

1                        True
volume                  False
log_return               True
mesa_open               False
open_xsinx2             False
                        ...  
%low_28 low_sma30       False
%low_28 %low_sma30      False
low_sma30^2             False
low_sma30 %low_sma30    False
%low_sma30^2            False
Length: 1431, dtype: bool

In [117]:
X4_poly = X3_poly[X3_poly.columns[pmask5]]

In [119]:
X4_poly.shape

(8782, 536)

In [118]:
model6 = sm.OLS(y, X4_poly)
results6 = model6.fit()
results6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.353
Model:                            OLS   Adj. R-squared:                  0.325
Method:                 Least Squares   F-statistic:                     12.33
Date:                Mon, 07 Sep 2020   Prob (F-statistic):               0.00
Time:                        19:28:01   Log-Likelihood:                 40372.
No. Observations:                8782   AIC:                        -8.000e+04
Df Residuals:                    8408   BIC:                        -7.735e+04
Df Model:                         373                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
1                                   -0.1853      0.157     -1.177      0.239      -0.494       0.123
log_return                          76.6337     14.303      5.358      0.000      48.596     104.671
open_xtanx2                          0.2922      0.873      0.335      0.738      -1.420       2.004
high_angle                           0.0001      0.000      0.726      0.468      -0.000       0.000
high_xsinxcosx                    1995.2816   1216.059      1.641      0.101    -388.493    4379.056
high_xsinx2                      -1383.8346    553.743     -2.499      0.012   -2469.308    -298.361
high_xsinxcosx2                    326.7316    357.655      0.914      0.361    -374.361    1027.824
high_xtanx2                         -3.4685      0.483     -7.179      0.000      -4.416      -2.521
mesa_low                            -0.1289      0.193     -0.668      0.504      -0.507       0.249
low_sin                            -21.0667      7.564     -2.785      0.005     -35.894      -6.239
%close                             -76.9900     14.305     -5.382      0.000    -105.031     -48.949
mesa_close                           0.6342      0.323      1.964      0.050       0.001       1.267
close_xcosx2                        42.4841      6.238      6.810      0.000      30.255      54.713
%open_9                              1.0708      5.253      0.204      0.838      -9.227      11.369
%open_sma30                          1.1242      1.348      0.834      0.404      -1.518       3.767
%close_3                            -0.2723      0.173     -1.578      0.115      -0.611       0.066
%close_21                            4.2601      2.852      1.494      0.135      -1.331       9.851
close_sma30                          0.0263      0.010      2.689      0.007       0.007       0.045
%high_23                             3.8161      2.279      1.675      0.094      -0.651       8.283
%low_22                              2.5700      1.701      1.511      0.131      -0.764       5.904
%low_28                              3.7386      2.041      1.831      0.067      -0.263       7.740
volume mesa_open                  1.888e-07   2.99e-08      6.317      0.000     1.3e-07    2.47e-07
volume open_xsinxcosx2              -0.0004      0.000     -2.466      0.014      -0.001   -7.98e-05
volume high_phase                 2.635e-13   5.67e-12      0.047      0.963   -1.08e-11    1.14e-11
volume %low                      -9.178e-07   1.23e-06     -0.749      0.454   -3.32e-06    1.48e-06
volume close_xcosx2               5.848e-06   1.63e-06      3.584      0.000    2.65e-06    9.05e-06
volume close_xsinxcosx2             -0.0001      0.000     -0.640      0.522      -0.001       0.000
volume %open_4                   -4.072e-06   1.12e-0

In [121]:
pmask6 = results6.pvalues < 0.05
X5 = X4_poly[X4_poly.columns[pmask6]]

model7 = sm.OLS(y, X5)
results7 = model7.fit()
results7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 target   R-squared (uncentered):                   0.273
Model:                            OLS   Adj. R-squared (uncentered):              0.255
Method:                 Least Squares   F-statistic:                              15.41
Date:                Mon, 07 Sep 2020   Prob (F-statistic):                        0.00
Time:                        19:31:02   Log-Likelihood:                          39855.
No. Observations:                8782   AIC:                                 -7.929e+04
Df Residuals:                    8573   BIC:                                 -7.781e+04
Df Model:                         209                                                  
Covariance Type:            nonrobust                                                  
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
log_return                          81.2088     11.181      7.263      0.000      59.291     103.127
high_xsinx2                         13.7569    165.397      0.083      0.934    -310.461     337.975
high_xtanx2                         -2.6330      0.295     -8.937      0.000      -3.211      -2.055
low_sin                            -15.0841      4.880     -3.091      0.002     -24.651      -5.517
%close                             -81.3812     11.190     -7.273      0.000    -103.316     -59.446
mesa_close                          -0.0002      0.000     -0.486      0.627      -0.001       0.001
close_xcosx2                        28.2443      3.777      7.479      0.000      20.841      35.647
close_sma30                          0.0025      0.001      2.995      0.003       0.001       0.004
volume mesa_open                  2.025e-07   2.94e-08      6.889      0.000    1.45e-07     2.6e-07
volume open_xsinxcosx2              -0.0003   9.84e-05     -3.005      0.003      -0.000      -0.000
volume close_xcosx2               4.462e-06   1.31e-06      3.418      0.001     1.9e-06    7.02e-06
volume %open_4                    -1.34e-06   1.01e-06     -1.326      0.185   -3.32e-06    6.41e-07
volume %open_42                   1.301e-06   6.31e-07      2.061      0.039    6.37e-08    2.54e-06
volume %close_21                 -3.841e-06   8.05e-07     -4.773      0.000   -5.42e-06   -2.26e-06
volume close_sma30               -2.629e-07   1.87e-07     -1.409      0.159   -6.29e-07    1.03e-07
volume %high_22                  -4.359e-06    1.2e-06     -3.644      0.000    -6.7e-06   -2.01e-06
volume %low_22                    4.645e-06   1.24e-06      3.754      0.000    2.22e-06    7.07e-06
volume low_sma30                  2.573e-07   1.86e-07      1.380      0.168   -1.08e-07    6.23e-07
log_return high_xsinx2            5954.1237   2.33e+04      0.255      0.799   -3.98e+04    5.17e+04
log_return low_xcosx              1257.7820    407.269      3.088      0.002     459.438    2056.126
log_return low_xsinxcosx2        -2.235e+04   2.34e+04     -0.955      0.339   -6.82e+04    2.35e+04
log_return low_xtanx2               91.8457     26.075      3.522      0.000      40.733     142.958
log_return close_xsinxcosx       -5564.5120   2.11e+04     -0.264      0.792   -4.69e+04    3.57e+04
log_return open_kurt50               3.4463      0.478      7.213      0.000       2.510       4.383
log_return %close_21             -4087.7288    879.719     -4.647      0.000   -5812.189   -2363.268
log_return close_kurt50             -2.0511      0.293     -6.990      0.000      -2.626      -1.476
log_return %high_21               4569.5654    971.761 

In [122]:
X5.shape

(8782, 224)

In [120]:
lr = LogisticRegression()

lr.fit(X4_poly, ys)
lr.score(X4_poly, ys)

0.5603507173764518

In [123]:
lr = LogisticRegression()

lr.fit(X5, ys)
lr.score(X5, ys)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.5604645866545206

In [125]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X5, y)
lr.score(X5, y)

0.26929560554580634

In [126]:
X5

,log_return,high_xsinx2,high_xtanx2,low_sin,%close,mesa_close,close_xcosx2,close_sma30,volume mesa_open,volume open_xsinxcosx2,volume close_xcosx2,volume %open_4,volume %open_42,volume %close_21,volume close_sma30,volume %high_22,volume %low_22,volume low_sma30,log_return high_xsinx2,log_return low_xcosx,log_return low_xsinxcosx2,log_return low_xtanx2,log_return close_xsinxcosx,log_return open_kurt50,log_return %close_21,log_return close_kurt50,log_return %high_21,log_return low_sma30,mesa_open high_sin2,mesa_open high_xsinxcosx,mesa_open %low_3,open_xsinx2 high_angle,open_xsinx2 high_xsinxcosx,open_xsinx2 high_xsinx2,open_xsinx2 high_xtanx2,open_xsinx2 close_xsinxcosx,open_xsinx2 close_xcosx2,open_xsinx2 close_xsinxcosx2,open_xsinx2 %open_4,open_xsinx2 %open_43,open_xsinx2 %open_sma30,open_xsinx2 %close_2,open_xsinx2 %close_21,open_xsinx2 close_sma30,open_xsinx2 close_kurt50,open_xsinx2 %high_21,open_xsinx2 %low_sma30,open_xsinxcosx2 high_xsinxcosx,open_xsinxcosx2 high_xsinxcosx2,open_xsinxcosx2 close_xsinxcosx,open_xsinxcosx2 close_xsinxcosx2,open_xsinxcosx2 open_kurt50,open_xsinxcosx2 %close_3,open_xsinxcosx2 close_kurt50,open_xtanx2 close_xcosx2,open_xtanx2 %open_9,open_xtanx2 %close_1,open_xtanx2 %close_2,open_xtanx2 %high_21,high_phase high_xsinxcosx,high_phase high_xsinxcosx2,high_phase low_xsinxcosx2,high_phase close_xcosx,high_phase close_xcosx2,high_phase %open_43,high_phase open_kurt50,high_phase close_kurt50,high_angle high_xsinxcosx2,high_angle %low_23,high_sin2 low_xcosx,high_sin2 low_xsinxcosx2,high_sin2 %open_4,high_sin2 %open_9,high_sin2 %open_sma30,high_xsinxcosx high_xsinxcosx2,high_xsinxcosx %low,high_xsinxcosx low_sin,high_xsinxcosx low_xtanx2,high_xsinxcosx close_xsinxcosx,high_xsinxcosx %open_42,high_xsinxcosx %open_sma30,high_xsinx2 %low,high_xsinx2 mesa_low,high_xsinx2 low_sin,high_xsinx2 low_xtanx2,high_xsinx2 %close,high_xsinx2 mesa_close,high_xsinx2 close_xsinxcosx,high_xsinx2 close_xsinxcosx2,high_xsinx2 %open_41,high_xsinx2 %open_43,high_xcosx2 high_xtanx2,high_xcosx2 low_sin,high_xcosx2 close_xcosx,high_xcosx2 close_xcosx2,high_xcosx2 %open_4,high_xcosx2 %close_1,high_xcosx2 %close_2,high_xcosx2 close_kurt50,high_xcosx2 %high_21,...,%low^2,%low low_xcosx,%low close_xcosx2,%low open_kurt50,%low %open_sma30,%low %low_sma30,mesa_low low_sin,mesa_low close_xcosx2,mesa_low %open_4,mesa_low %close_2,low_sin low_xsinx2,low_sin low_xtanx2,low_sin %low_3,low_sin %low_sma30,low_tan2 low_xsinxcosx2,low_tan2 close_xcosx,low_tan2 close_xsinxcosx,low_tan2 close_xcosx2,low_xcosx^2,low_xcosx low_xsinxcosx2,low_xcosx %close,low_xcosx open_kurt50,low_xcosx %open_sma30,low_xcosx %low_sma30,low_xsinx2 low_xtanx2,low_xsinx2 close_xcosx,low_xsinx2 close_xcosx2,low_xsinx2 %open_4,low_xsinx2 close_sma30,low_xsinxcosx2 %close,low_xsinxcosx2 %open_41,low_xsinxcosx2 %close_1,low_xtanx2 %close,low_xtanx2 close_xsinxcosx,low_xtanx2 %open_4,low_xtanx2 open_kurt50,%close close_xsinxcosx,%close open_kurt50,%close %close_21,%close close_sma30,%close close_kurt50,%close %high_21,mesa_close %open_4,mesa_close %low_3,mesa_close low_sma30,close_sin2 %close_1,close_xcosx %open_9,close_xcosx %open_42,close_xcosx %open_43,close_xcosx open_kurt50,close_xcosx %close_1,close_xcosx close_sma30,close_xcosx %high_22,close_xcosx %low_28,close_xcosx low_sma30,close_xsinxcosx %open_9,close_xsinxcosx open_kurt50,close_xsinxcosx %low_22,close_xsinxcosx %low_23,close_xsinxcosx %low_sma30,close_xcosx2 %open_9,close_xcosx2 %open_42,close_xcosx2 %open_43,close_xcosx2 open_kurt50,close_xcosx2 %open_sma30,close_xcosx2 close_kurt50,close_xcosx2 %low_sma30,close_xsinxcosx2 %low_3,%open_4^2,%open_4 %open_9,%open_4 open_kurt50,%open_4 %close_1,%open_4 %high_23,%open_4 %low_sma30,%open_9^2,%open_9 %low_3,%open_41 %high_24,%open_42 %close_1,%open_43^2,open_kurt50 %close_21,open_kurt50 %high_21,open_kurt50 %high_23,open_kurt50 %high_24,open_kurt50 %low_23,%open_sma30 %close_3,%open_sma30 %close_21,%close_1 %high_23,%close_1 %high_24,%close_1 %low_3,%close_2 %

In [129]:
pmask8 = results7.pvalues < 0.05
X6 = X5[X5.columns[pmask8]]
X6

,log_return,high_xtanx2,low_sin,%close,close_xcosx2,close_sma30,volume mesa_open,volume open_xsinxcosx2,volume close_xcosx2,volume %open_42,volume %close_21,volume %high_22,volume %low_22,log_return low_xcosx,log_return low_xtanx2,log_return open_kurt50,log_return %close_21,log_return close_kurt50,log_return %high_21,log_return low_sma30,mesa_open high_sin2,open_xsinx2 high_angle,open_xsinx2 high_xsinxcosx,open_xsinx2 close_xcosx2,open_xsinx2 close_xsinxcosx2,open_xsinx2 %open_4,open_xsinx2 %close_21,open_xsinx2 close_sma30,open_xsinx2 %high_21,open_xsinxcosx2 open_kurt50,open_xsinxcosx2 close_kurt50,open_xtanx2 close_xcosx2,high_phase high_xsinxcosx,high_phase high_xsinxcosx2,high_phase low_xsinxcosx2,high_phase open_kurt50,high_phase close_kurt50,high_angle high_xsinxcosx2,high_angle %low_23,high_sin2 low_xcosx,high_sin2 low_xsinxcosx2,high_sin2 %open_9,high_sin2 %open_sma30,high_xsinxcosx high_xsinxcosx2,high_xsinxcosx low_xtanx2,high_xsinxcosx %open_sma30,high_xsinx2 %low,high_xsinx2 mesa_low,high_xsinx2 mesa_close,high_xsinx2 %open_41,high_xcosx2 low_sin,high_xcosx2 close_xcosx,high_xcosx2 close_xcosx2,high_xcosx2 %open_4,high_xcosx2 %close_1,high_xcosx2 %close_2,high_xcosx2 %high_21,high_xsinxcosx2 low_xtanx2,high_xsinxcosx2 close_sin2,high_xsinxcosx2 %open_43,high_xsinxcosx2 open_kurt50,high_xsinxcosx2 %close_1,high_xsinxcosx2 %low_3,high_xsinxcosx2 %low_sma30,high_xtanx2 %low,high_xtanx2 mesa_low,high_xtanx2 low_xcosx,high_xtanx2 low_xtanx2,high_xtanx2 mesa_close,high_xtanx2 %open_4,high_xtanx2 %close_2,high_xtanx2 close_kurt50,high_xtanx2 %high_22,high_xtanx2 %low_3,high_xtanx2 %low_22,high_xtanx2 %low_23,%low^2,%low low_xcosx,%low close_xcosx2,%low open_kurt50,%low %open_sma30,%low %low_sma30,mesa_low low_sin,mesa_low close_xcosx2,mesa_low %open_4,low_sin low_xtanx2,low_sin %low_3,low_xcosx^2,low_xcosx %close,low_xcosx open_kurt50,low_xcosx %open_sma30,low_xcosx %low_sma30,low_xsinx2 close_xcosx,low_xsinx2 close_xcosx2,low_xsinx2 %open_4,low_xsinxcosx2 %close_1,low_xtanx2 %close,low_xtanx2 close_xsinxcosx,low_xtanx2 %open_4,low_xtanx2 open_kurt50,%close open_kurt50,%close %close_21,%close close_sma30,%close close_kurt50,%close %high_21,mesa_close %open_4,mesa_close low_sma30,close_sin2 %close_1,close_xcosx %open_9,close_xcosx %open_42,close_xcosx %open_43,close_xcosx open_kurt50,close_xcosx %close_1,close_xcosx close_sma30,close_xcosx %high_22,close_xcosx %low_28,close_xcosx low_sma30,close_xsinxcosx %open_9,close_xsinxcosx %low_22,close_xsinxcosx %low_sma30,close_xcosx2 %open_9,close_xcosx2 %open_42,close_xcosx2 %open_43,close_xcosx2 open_kurt50,close_xcosx2 %open_sma30,close_xcosx2 %low_sma30,%open_4^2,%open_4 %open_9,%open_4 open_kurt50,%open_4 %close_1,%open_4 %high_23,%open_4 %low_sma30,%open_9^2,%open_9 %low_3,%open_41 %high_24,%open_42 %close_1,%open_43^2,open_kurt50 %close_21,open_kurt50 %high_21,open_kurt50 %high_23,open_kurt50 %high_24,open_kurt50 %low_23,%open_sma30 %close_21,%close_1 %high_24,%close_1 %low_3,%close_2 %high_21,close_sma30 close_kurt50,close_kurt50^2,close_kurt50 %high_21,close_kurt50 %high_24,close_kurt50 low_sma30,%high_21 %low_28,%high_22 %high_24
8844,-0.008060,0.004905,0.001151,-0.008028,0.007855,17.374333,97.986868,0.000143,1.571062,-0.118168,-0.229621,0.000000,-0.229358,0.000019,0.000048,0.011474,9.253636e-06,0.008020,1.385659e-05,-0.139907,-0.000702,0.017548,-6.593834e-12,5.637702e-09,-4.989036e-12,6.691779e-09,-8.239877e-10,0.000012,-1.233857e-09,-1.020333e-06,-7.131613e-07,-1.172115e-06,3.556144e-03,3.801288e-03,0.000911,551.016398,385.132724,-0.240121,0.000000,3.300078e-06,3.374866e-09,-0.000000e+00,5.147068e-06,9.022972e-11,5.517104e-08,3.298461e-08,-3.316650e-08,5.638545e-06,5.621956e-06,-6.803888e-09,7.885193e-06,4.346880e-05,5.381039e-05,6.387128e-05,3.553415e-05,1.186523e-05,-1.177684e-05,5.897428e-08,8.690907e-09,5.799163e-09,1.398089e-05,-5.094406e-08,-0.000000e+00,3.300345e-08,-1.413403e-05,0.002403,-1.128943e-05,-2.945152e-05,0.002396,4.572969e-05,8.495109e-06,-0.004880,

In [130]:
model8 = sm.OLS(y, X6)
results8 = model8.fit()
results8.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 target   R-squared (uncentered):                   0.247
Model:                            OLS   Adj. R-squared (uncentered):              0.234
Method:                 Least Squares   F-statistic:                              18.50
Date:                Mon, 07 Sep 2020   Prob (F-statistic):                        0.00
Time:                        19:39:08   Log-Likelihood:                          39700.
No. Observations:                8782   AIC:                                 -7.909e+04
Df Residuals:                    8629   BIC:                                 -7.801e+04
Df Model:                         153                                                  
Covariance Type:            nonrobust                                                  
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
log_return                        71.2283      9.960      7.151      0.000      51.704      90.752
high_xtanx2                       -2.5979      0.292     -8.897      0.000      -3.170      -2.026
low_sin                           -9.3079      3.835     -2.427      0.015     -16.825      -1.791
%close                           -71.4231      9.967     -7.166      0.000     -90.961     -51.886
close_xcosx2                      19.9360      3.464      5.755      0.000      13.145      26.727
close_sma30                        0.0023      0.001      3.809      0.000       0.001       0.003
volume mesa_open                1.731e-08   4.03e-09      4.296      0.000    9.41e-09    2.52e-08
volume open_xsinxcosx2            -0.0003   4.93e-05     -5.321      0.000      -0.000      -0.000
volume close_xcosx2              3.18e-06   1.24e-06      2.563      0.010    7.48e-07    5.61e-06
volume %open_42                     1e-06   6.36e-07      1.574      0.116   -2.46e-07    2.25e-06
volume %close_21               -3.996e-06    7.9e-07     -5.057      0.000   -5.55e-06   -2.45e-06
volume %high_22                -3.447e-06   1.17e-06     -2.943      0.003   -5.74e-06   -1.15e-06
volume %low_22                  3.518e-06   1.18e-06      2.970      0.003     1.2e-06    5.84e-06
log_return low_xcosx            1835.0284    307.226      5.973      0.000    1232.791    2437.265
log_return low_xtanx2             64.7215     11.140      5.810      0.000      42.884      86.559
log_return open_kurt50             3.0836      0.372      8.298      0.000       2.355       3.812
log_return %close_21           -3431.4560    770.860     -4.451      0.000   -4942.525   -1920.387
log_return close_kurt50           -1.3556      0.223     -6.065      0.000      -1.794      -0.917
log_return %high_21             3722.7607    850.322      4.378      0.000    2055.926    5389.596
log_return low_sma30              -4.4763      0.673     -6.655      0.000      -5.795      -3.158
mesa_open high_sin2                0.0023      0.005      0.486      0.627      -0.007       0.012
open_xsinx2 high_angle            -0.0003   8.49e-05     -3.066      0.002      -0.000   -9.38e-05
open_xsinx2 high_xsinxcosx      3.506e+04   2.47e+04      1.419      0.156   -1.34e+04    8.35e+04
open_xsinx2 close_xcosx2        -575.4710    120.897     -4.760      0.000    -812.459    -338.483
open_xsinx2 close_xsinxcosx2    2179.8524    331.553      6.575      0.000    1529.929    2829.776
open_xsinx2 %open_4              -55.7726    130.424     -0.428      0.669    -311.435     199.890
open_xsinx2 %close_21           -135.6642    209.634     -0.647      0.518    -546.597     275.268
open_xsinx2 cl